<a href="https://colab.research.google.com/github/samrudhi93/Machine-Translation-techniques/blob/main/Seq2Seq_machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget /content/drive/MyDrive/Colab_Notebooks/fra.txt

/content/drive/MyDrive/Colab_Notebooks/fra.txt: Scheme missing.


In [2]:
filename = '/content/drive/MyDrive/Colab_Notebooks/fra.txt'
txt_file = open(filename, 'r')
FileContent = txt_file.read(300)
FileContent

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)\nGo.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)\nGo.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)\nHi.\tSalut !\tCC-BY 2.0 ('

**Data Preprocessing**

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Dense

In [4]:
import numpy as np
import pandas as pd


In [5]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
FileContent

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)\nGo.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)\nGo.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)\nHi.\tSalut !\tCC-BY 2.0 ('

#### Preparing data

In [6]:
with open('/content/drive/My Drive/Colab_Notebooks/fra.txt', 'r') as f:
  lines = f.read().split("\n")

In [7]:
lines

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)',
 'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba

In [37]:
#vectorise the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open('/content/drive/My Drive/Colab_Notebooks/fra.txt', 'r', encoding='utf-8') as f:
  lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")  # /t is used 2 times so for that we use '_' to split it and we get input and targets 
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)
    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)
#Storing unique characters present in the text 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


#Now to get the sequnece length of the text we take the max of it. the length of text is from 11 to 15. so here it take max of it which is 15
max_encoder_sequence_length = max([len(txt) for txt in input_texts]) 
max_decoder_sequence_length = max([len(txt) for txt in target_texts])
max_decoder_sequence_length

59

In [43]:
len(input_texts)
for i in target_texts:
  print(len(i))

TypeError: ignored

##Test code

In [12]:
para = 'i am recognizing the pattern in my behaviour. \thu mari swabhav ne odkhu chu. \tCC-BY 2.0 (France) Attribution: tatoeba.org #5828662 (CK) & #4678647 (sacredceltic)'

In [32]:
sampl = 150
inp = []
out= []
input_char = set()
output_char = set()
for i in para[: min(sampl, len(para) - 1)]:
  input_tex, target_tex, _ = para.split("\t")
  target_tex = "/t" + target_tex + "\n"
  inp.append(input_tex)
  out.append(target_tex)
  for char in input_tex:
    if char not in input_char:
      input_char.add(char)
  for char in target_tex:
    if char not in output_char:
      output_char.add(char)  
      #Now we have a single set of characters stored in input_char and output_char 

input_char = sorted(list(input_char)) 
output_char = sorted(list(output_char))
max_encoder_seq_length = max([len(txt) for txt in inp])
print(max_encoder_seq_length)    

46


In [33]:
len(inp)

150